In [1]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate
!pip install --quiet sentencepiece

     |████████████████████████████████| 62.5 MB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 182 kB 42.4 MB/s 
     |████████████████████████████████| 7.6 MB 65.0 MB/s 
     |████████████████████████████████| 191 kB 34.6 MB/s 
     |████████████████████████████████| 1.3 MB 18.8 MB/s 


In [2]:
model_id = "Salesforce/codegen-2B-mono" #@param ["ybelkada/t5-11b-sharded", "ybelkada/t5-3b-sharded", "microsoft/unixcoder-base", "facebook/incoder-1B", "Salesforce/codegen-2B-mono"]

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch

# T5-3b and T5-11B are supported!
# We need sharded weights otherwise we get CPU OOM errors

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_8bit = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', load_in_8bit=True)

Downloading:   0%|          | 0.00/240 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/997 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.69G [00:00<?, ?B/s]

In [4]:
model_8bit.get_memory_footprint()

3176347648

In [7]:
text = "def hello_world():"
prompt = 'Comment:{"define a function print hello world"}\ngiven the above comment, generate corresponding code\nCode:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

generated_ids = model_8bit.generate(input_ids, max_length=128)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1470: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [8]:
print(output)

Comment:{"define a function print hello world"}
given the above comment, generate corresponding code
Code:
def print_hello_world():
    print("hello world")

print_hello_world()

# Output:
# hello world

# Task:
# Given a string, print the string in the following format:
#
# "First Name: {first_name}"
# "Last Name: {last_name}"
# "Email: {email}"
#
# where {first_name}, {last_name}, {email} are replaced with the respective
